In [1]:
import time
import torch
import torch.nn as nn
from torchvision.datasets import MNIST
from torch.optim import RMSprop
from torchvision import transforms
from torch.utils.data import DataLoader

C:\python39\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (5.0.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
start_time = time.time()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))]) # data preprocessing ve augmentasyon islemlerini birlestirmek icin kullanilir.

In [4]:
train_dataset = MNIST(root='', train=True, download=False, transform=transform)
test_dataset = MNIST(root='', train=False, download=True, transform=transform)

In [5]:
train_data_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_data_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

In [6]:
class SimpleAnn(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu1 = nn.ReLU()
        
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.relu2 = nn.ReLU()
        
        self.fc3 = nn.Linear(hidden_dim, output_dim)
        self.log_softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, x):
        out = x.view(-1, 28*28)
        
        out = self.fc1(out)
        out = self.relu1(out)
        
        out = self.fc2(out)
        out = self.relu2(out)
        
        out = self.fc3(out)
        out = self.log_softmax(out)
        
        return out

In [7]:
input_dim = 28*28
hidden_dim = 128
output_dim = 10

In [8]:
model = SimpleAnn(input_dim=input_dim,
                 hidden_dim=hidden_dim,
                 output_dim=output_dim).to(device)

In [9]:
criterion = nn.CrossEntropyLoss()

lr = 1e-4
optimizer = RMSprop(model.parameters(), lr=lr, momentum=0.9)

In [10]:
num_epoch = 30
count = 0
loss_list = []

In [11]:
for epoch in range(num_epoch):
    for images, labels in train_data_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
    print(f"Epoch {epoch+1}/{num_epoch}, Loss: {loss.item()}")

Epoch 1/30, Loss: 0.17780403792858124
Epoch 2/30, Loss: 0.2518538236618042
Epoch 3/30, Loss: 0.14699581265449524
Epoch 4/30, Loss: 0.09763103723526001
Epoch 5/30, Loss: 0.14549218118190765
Epoch 6/30, Loss: 0.3091594874858856
Epoch 7/30, Loss: 0.219527468085289
Epoch 8/30, Loss: 0.004463255405426025
Epoch 9/30, Loss: 0.022847840562462807
Epoch 10/30, Loss: 0.06613457202911377
Epoch 11/30, Loss: 0.0019192604813724756
Epoch 12/30, Loss: 0.0007478297920897603
Epoch 13/30, Loss: 0.002960864920169115
Epoch 14/30, Loss: 0.0034197031054645777
Epoch 15/30, Loss: 0.0008968302863650024
Epoch 16/30, Loss: 0.0022288665641099215
Epoch 17/30, Loss: 0.22881759703159332
Epoch 18/30, Loss: 0.0003365853917784989
Epoch 19/30, Loss: 0.0035546571016311646
Epoch 20/30, Loss: 0.27667224407196045
Epoch 21/30, Loss: 0.002561437664553523
Epoch 22/30, Loss: 0.03764021024107933
Epoch 23/30, Loss: 5.094634980196133e-05
Epoch 24/30, Loss: 0.023407092317938805
Epoch 25/30, Loss: 0.0010568887228146195
Epoch 26/30, Lo

In [ ]:
model.eval() # pytorch modelini degerlendirme moduna getirir. Bu ozellikle test ve degerlendirme sirasinda kullanislidir.
# Modelin train sirasinda ogrenilen ozelliklerini kullanmasini birakmasina ve test surecinde tutarli sonuclar uretmesine
# yardimci olur.
# Bu, ozellikle dropout veya batch normalization gibi train sirasinda kullanilan tekniklerin test sirasinda devre disi
# birakilmasi gerektigi durumlar icin gecerlidir.

correct = 0
total = 0

In [ ]:
with torch.no_grad():
    for images, labels in test_data_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

In [ ]:
accuracy = correct / total
print(f"Test Accuracy: % {accuracy * 100}")

In [ ]:
end_time = time.time()
print(end_time - start_time)